# Testing

We begin with a simple example. We want to implement a function which computes the allowed magnetic quantum numbers $m$ for any given angular momentum $j$. Remember that $m = -j, -j+1, \ldots , j-1, j$. This function is implemented below.

<div class="alert alert-block alert-warning">
    <b>HINT</b>
    <br> Most examples in this notebook contain an implementation of the function that we want to test. We will always have a set of requirements for each function and out tests are meant to ensure that those requirements are met. Reading the actual implementations might bias our tests.
</div>

<div class="alert alert-block alert-success">
    <b>NOTE</b>
    <br>The code samples are not always written in a clean way but are sometimes overly convoluted or complicated in order to garnish them with hidden bugs. If you are looking for advice on how to write good Python, search elsewhere!
</div>

In [ ]:
import numpy as np

In [ ]:
def magnetic_quantum_numbers(j):
    """
    Returns a list of all magnetic quantum number for a given angular momentum j.
    The results are sorted in ascending order.
    """
    if j < 0:
        raise ValueError('j must be greater equal 0')
    negative = np.arange(-j, 0)
    positive = -negative[::-1]
    return list(np.concatenate((negative, [0], positive)))

In order to test tis function, we can compare the output to what we would expect. For example:
- $j = 0 \rightarrow m = 0$
- $j = 1 \rightarrow m = 0, \pm1$

According to its docstring, `magnetic_quantum_numbers` promises to return a list of numbers sorted in ascending order. This implies that for $j=0,1$, we expect to get `[0]` and `[-1, 0, 1]`.

We can encode tests for these by using assertions. The `assert` keyword in Python takes a condition and if that condition is false, raises an `AssertionError`.

In [ ]:
assert magnetic_quantum_numbers(0) == [0]
assert magnetic_quantum_numbers(1) == [-1, 0, 1]

No output was produced which means that both assertions passed. Our tests were successful! If this is too boring for you, feel free to add a print statement at the end of a cell to notify you that the tests passed, see below.

Those tests are great and all, but only two tests are hardly enough. In particular, we have only used integer values for $j$. Let's try a half integer now:

In [ ]:
assert magnetic_quantum_numbers(0.5) == [-0.5, 0.5]
print("Success!")

This is not so great anymore. What you are seing above is one of the aforementioned `AssertionErrors`. Unfortunately, it does not provide any useful information other than letting us know that the test failed. We will later see ways for getting better outputs for failed tests. For now, just look at the result of our function manually:

In [ ]:
magnetic_quantum_numbers(0.5)

There is an extra 0 in the list. This was fine for integer $j$ but it should not be there for half integers.
We will fix `magnetic_quantum_numbers` later, but for now we leave it as it is.

### Error Modes

Until now, we have only looked at the 'happy path' of our function, meaning that all inputs are valid and the computation works, albeit sometimes incorrectly. But it is equally important to test the error paths. This is to ensure that errors such as invalid inputs or failure of intermediate operations are detected and handled properly. When that is not the case, the results of a function can be incorrect without anyone noticing.

***
#### Aside: Exceptions
In Python, errors are almost always reported in the form of exceptions. You do not need to understand exactly how the work now. In order to signal ('raise') an error, we can simply write
```python
raise RuntimeError("message")
```
We can pass any message we want to describe the error. There are multiple predefined error types in Python. `RuntimeError` is a general error that we can use in most situations. When a function argument is invalid, one usually raises a `ValueError`.
***

`magnetic_quantum_numbers` has two conditions on its input:
- $j \geq 0$
- $j$ is an integer or half integer

We thus expect our function to raise a `ValueError` if either condition is violated.
In order to test for that we use the external package __[pytest](https://docs.pytest.org/en/stable/index.html)__ which provides many utilities for writing tests. Here, we need the contexr manager `pytest.raises`. Again, there is no need to understand what a context manager is. For our purposes, simply read
```python
with pytest.raises(ValueError):
```
as 'check that the indented code below it raises a `ValueError`'.

In [ ]:
import pytest
with pytest.raises(ValueError):
    magnetic_quantum_numbers(-1)
with pytest.raises(ValueError):
    magnetic_quantum_numbers(0.2)

Another failure! But look at the error message. Only the second test failed by not raising a `ValueError`. The first one succeded. Looking at the implementation of `magnetic_quantum_numbers` above shows that there is indeed a check $j < 0$ but none for the integer condition.

It is important to perform these kinds of tests or error modes. We cannot write a sensible test for the expected output of `magnetic_quantum_numbers(0.2)` because that is simply invalid. If this is not detected, however, a user might call it with an invalid argument and get garbage results without knowing it!

Below you can find a complete implementation which passes all of our tests. Feel free to try it out and write more tests.

In [ ]:
def magnetic_quantum_numbers(j):
    if j < 0:
        raise ValueError('j must be greater equal 0')
    if (2 * j) != int(2 * j):
        raise ValueError('j must be an integer or half integer')
    return list(np.arange(-j, j+1, 1))

### Zeeman Splitting

We can now use our function to compute physical observables. As an example, we are going to look at the Zeeman effect. Lt's start by reminding ourselves of the formula. The coupling of the spin of an electron to an external magentic field $B$ shifts the energy level of that electron by
$$
\Delta E = \mu_B B g_j m_j ,
$$
where $\mu_B$ is the Bohr Magneton, $g_j$ the Landé factor, and $m_j$ the magnetic quantum number that we computed above. For simplicity, we only look at energy shifts relative to the magnetic field and omit $B$ in the following.

First, we define a function which computes all energy shifts for a given set of angular momenta:

In [ ]:
def zeeman_shifts(j, l, s):
    lande = 1 + (j*(j+1) - l*(l+1) + s*(s+1)) / (2*j*(j+1))
    magneton = physical_constants['Bohr magneton in eV/T'][0]
    magn = magnetic_quantum_numbers(j)
    return [magneton * m * lande for m in magn]

We are now faced with a difficult problem. We have an piece of Python code which implements a mathematical equation. But how can we test whether those two match?
Before, we had a simple rule for what the output should be. This let us write some tests to compare the actual to the expected output. However, now it seems as though we need the output of our function in order to know what result it should produce in the first place.

Luckily though, people have performed many calculations based on the Zeeman effect already. So we can pick one or more of those results and compare out implementation with them.

Let us choose the Lyman-α line, meaning the transition of an electron from an $n = 2$ orbital to $n=1$ in hydrogen. The impact of the Zeeman effect on this transition depends on the angular momentum, of course.
For a transition from $|n, l, j, m_j\rangle = |2, 1, \frac{1}{2}, +\frac{1}{2}\rangle$ to $|1, 0, \frac{1}{2}, +\frac{1}{2}\rangle$, the Zeeman effect induced energy difference is $- \frac{2}{3} \mu_B$. So let's use that as a test: (We have electrons, so $s = \frac{1}{2}$ is fixed.)

In [ ]:
from scipy.constants import physical_constants
# The `[0]` in physical_constants['name'][0] extracts the actual value.
bohr_magneton = physical_constants['Bohr magneton in eV/T'][0]
before = zeeman_shifts(1/2, 1, 1/2)
after = zeeman_shifts(1/2, 0, 1/2)
assert before[1]-after[1] == -2 / 3 * bohr_magneton

The test passes! Note that we are using `before[1]` and `after[1]`, which extract the $m_j = +\frac{1}{2}$ elements according to

In [ ]:
magnetic_quantum_numbers(1/2)

As a nice bonus, we are loading the Bohr magneton from SciPy with a specific unit. This lets us check if our function uses the units we expect.

There are a lot more transitions we can use. For instance, $|2, 1, \frac{3}{2}, +\frac{1}{2}\rangle$ to $|1, 0, \frac{1}{2}, -\frac{1}{2}\rangle$ has an energy of $\frac{5}{3} \mu_B$:

In [ ]:
before = zeeman_shifts(3/2, 1, 1/2)
after = zeeman_shifts(1/2, 0, 1/2)
assert before[2]-after[0] == 5/3*bohr_magneton

What happened this time? Again, the failure message does not give us a lot of information. So let's output the results ourselves:

In [ ]:
print(before[2]-after[0])
print(5/3*bohr_magneton)

Pretty close but not an exact match! The problem here is that we are performing calculations with floating point numbers. Those have limited precision and we cannot expect two equivalent calculations to produce exactly the same results unless they do  _exactly_ the same operations in _exactly_ the same order.

This means that we should not use the equality operator (`==`) as above but rather test for approximate equality. A useful tool for this is the 'testing' module of numpy (__https://numpy.org/doc/stable/reference/routines.testing.html__). It provides __[assert_almost_equal](https://numpy.org/doc/stable/reference/generated/numpy.testing.assert_almost_equal.html#numpy.testing.assert_almost_equal)__ which does what we need.

In [ ]:
np.testing.assert_almost_equal(before[2]-after[0], 5/3*bohr_magneton)

Now there is no output meaning that the test passes.

We could keep going and add more tests and you should do this in practice. But these kinds of tests are often only of limited use because we generally do not have known outputs that we can easily compare to.

A different approach (potentially supplementing the above) is searching for simple cases where we know the correct result. One such case would be a particle without spin. Here, the interaction with the magnetic field depends only on the orbital angular momentum and should thus give an energy shift of $m_j \mu_B$. And indeed, our implementation 

In [ ]:
j = 1  # l = j here
magn = magnetic_quantum_numbers(j)
for i in range(len(magn)):
    np.testing.assert_almost_equal(
        zeeman_shifts(j, j, 0)[i],
        magn[i] * physical_constants['Bohr magneton in eV/T'][0])

Yet another testing ansatz is searching for properties of the expected results that we can test without knowing the actual numerical values.
An example is the length of the output. We know that `zeeman_shifts` has to produce a list with the same number of elements as `magnetic_quantum_numbers`. This gives us an opportunity for testing with a broad range of different inputs.

Below, write tests comparing the lengths. You can use the simple `assert` here because lengths are integers and can compare exactly equal. You can simply list a bunch of possible inputs `j`, `l`, `s` or, for extra points, write a loop / loops that iterate through several different values. Note that not all combinations are physically allowed: $j = l+s, l+s-1, \ldots , |l-s|$

In [ ]:
# -- YOUR CODE HERE --

Another property is that $\Delta E$ is proportional to $m_j$. This is a simple relationship that we can exploit! We can do so in two ways:
- $m_j$ always appears in pairs with opposite sign except for $m_j=0$. The Zeeman shifts thus appear in pairs as well. We can write tests that match the corresponding elements. Mind the order of results produced by `magnetic_quantum_numbers`.
- Alternatively, we divide out $m_j$ which should give us the same number for every value returned by the same call of `zeeman_shifts`.

As an exercise, implement one or both of these tests. While the second approach is more thorough, the first one can still be a good exercise. It might help to convert the list returned by `zeeman_shifts` to a numpy array first.

In [ ]:
# -- YOUR CODE HERE --

A last test before we are done with the Zeeman effect. In the beginning we talked about how important it is to also test the error modes of our code. So let's do this! `zeeman_shifts` has the same conditions on `j` as `magnetic_quantum_numbers`:

In [ ]:
with pytest.raises(ValueError):
    zeeman_shifts(0.2, 1, 0)

In [ ]:
with pytest.raises(ValueError):
    zeeman_shifts(-1, 1, 0)

We see that `zeeman_shifts` inherited the check for values of $j$ that are not integers or half-integers. But passing in $j=-1$ triggers a different error. We can remedy the situation by adding a corresponding check to the beginning of `zeeman_shifts`. Feel free to do this as a exercise.

But there are more potential errors. As already mentioned, there are physical restrictions on the allowed function arguments: $j = l+s, l+s-1, \ldots , |l-s|$
It is evident from the source code of `zeeman_shifts` that those are not verified. As an exercise, you can implement such a check in `zeeman_shifts` and add a corresponding test here:

In [ ]:
# -- YOUR CODE HERE --

***
# Exercises

## Histogramming

Here, we have a simple function for building histograms. When testing it, remember to think of possible _edge_ cases.

In [ ]:
def histogram(data, bin_edges):
    """
    Construct a histogram of one dimensional data.
    
    Arguments:
      data: 1D array.
      bin_edges: (len=nbins+1) 1D array of bin edges.
                  Bin i is in range [bin_edges[i], bin_edges[i+1]].
    """
    hist = np.zeros(shape=[len(bin_edges)-1], dtype=int)
    for value in data:
        for i in range(1, len(bin_edges)):
            if value < bin_edges[i]:
                hist[i-1] += 1
                break
    return hist

In [ ]:
# -- YOUR CODE HERE --
edges = np.array([0, 1, 2, 3])
np.testing.assert_array_equal(histogram(np.array([]), edges),
                              [0, 0, 0])
np.testing.assert_array_equal(histogram(np.array([0, 0.1, 0.6, 2, 2.2, 0.2]), edges),
                              [4, 0, 2])
np.testing.assert_array_equal(histogram(np.array([4, 3]), edges),
                              [0, 0, 0])
np.testing.assert_array_equal(histogram(np.array([-0.5, 0.1, 1.2, 2.2, 2.2, 0.3]),
                                        np.array([-1, 1, 2, 3])),
                              [3, 1, 2])

# failing
# np.testing.assert_array_equal(histogram(np.array([-0.5, 0.1, 1.2, 2.2, 2.2, 0.3]),
#                                         edges),
#                                [0, 0, 0])
# with pytest.raises(ValueError):
#     histogram(np.array([0, 0.1, 0.6, 2, 2.2, 0.2]), 
#               np.array([0, 1, 3, 2]))

## Leap Years

A leap year is any year that is divisible by 4 unless it is also divisible by 100 and not by 400. The author of the function tried to be clever but is it acutally correct?

In [ ]:
def is_leapyear(year):
    """
    Return True if year is a leap year, False otherwise.
    """
    return year % 400 == year % 100 + year % 4 != 0

In [ ]:
# -- YOUR CODE HERE --
assert is_leapyear(4)
assert not is_leapyear(3)
assert is_leapyear(400)
assert not is_leapyear(100)
assert is_leapyear(2020)
assert is_leapyear(2000)
assert not is_leapyear(1000)

## Chi Squared

$$
\chi^2 = \sum_{i=1}^{N}\, \frac{{(y_{\text{model},\,i} - y_{\text{meas},\,i})}^2}{\sigma^{2}_{i}}
$$

In [ ]:
def chi_squared(model, meas, errors):
    return np.sum(((model - meas)/errors)**2)

In [ ]:
# -- YOUR CODE HERE --

## Least Squares Fit*

<div class="alert alert-block alert-info">
    <b>NOTE</b>
    <br>This is a potentially longer and more complicated exercise, feel free to skip it.
</div>

In this exercise, we are looking at fitting a parabola using the method of least squares. There is a fundamental difference to the previous exercises. Here, we generally cannot sensibly construct an expected output because the fit has to deal with randomess. We can still test a number of special cases, though.
Additionally, in cases like this one, it can make sense to do some (at least partially) manual tests. For example, we can perform a fit and plot the result to see if it makes sense.

In [ ]:
def fit_parabola(x, y_meas, errors, start_params):
    """
    Perform a least squares fit of a parabola.
    
    Arguments:
      x: Independent variable.
      y_meas: measured values of the dependent variable.
      errors: Uncertainties of the measured values.
      start_params: (len=3) Iniitla values for the parameters of the parabola.
      
    Returns:
      Best fit results for the parameters of the parabola.
    """
    X = np.array([np.ones_like(x), x, x**2]).T
    V = np.diag(1/errors**2)
    return np.linalg.inv(X.T @ V @ X) @ X.T @ V @ y_meas

In [ ]:
# -- YOUR CODE HERE --

In [ ]:




Possible task:
d-spacing of theta=pi/2 in our convention is d=lambda/2

In [ ]:
def d_spacing(tof, path_length, angle):
    return h * tof / (path_length * neutron_mass * 2 * np.sin(angle))
def de_broglie(tof, path_length):
    return h*tof / (neutron_mass * path_length)
tof = 123
path_length = 1000
angle = np.pi/2
assert d_spacing(tof, path_length, angle) == de_broglie(tof, path_length)/2
# assert angle=0 -> inf

## Inelastic Neutron Scattering: Energy Transfer

The energy transfer in inelastic scattering of neutrons is
- **Direct**:
$$
\Delta E_{\text{direct}} = E_i - \frac{m_{\text{n}} L_2^2}{2 (t - t_0)}\, , \quad t_0 = \sqrt{\frac{L_1^2 m_\text{n}}{E_i}}
$$

- **Indirect**:
$$
\Delta E_{\text{indirect}} = \frac{m_{\text{n}} L_1^2}{2 (t - t_0)} - E_f\, , \quad t_0 = \sqrt{\frac{L_2^2 m_\text{n}}{E_f}}
$$

where $E_i$, $E_f$ are the initial and final energy, respectively. $t$ is the time-of-flight and $L_1$ and $L_2$ are the lengths of the primary and secondary flight paths.

As usual, try to find special cases where the equations become simpler and you can easily predict the output. Also, write tests with incorrect / nonsense input to check if the functions reports errors properly.

In [ ]:
from scipy.constants import neutron_mass

In [ ]:
def energy_transfer(ei_or_ef, tof, L1, L2, mode):
    """
    Compute the energy transfer for inelastic neutron scattering.
    
    All units are SI.
    
    Arguments:
      ei_or_ef: In direct scattering: the initial energy.
                In indirect scattering: the final energy.
      tof: Time-of-flight.
      L1: Primary flight path.
      L2: Secondary flight path.
      mode: Either 'direct' or 'indirect'.
    """
    t0 = np.sqrt(L1**2 * neutron_mass / ei_or_ef)
    delta_t = tof - t0
    if mode == 'direct':
        return ei_or_ef - neutron_mass * L2**2 / 2 / delta_t**2
    elif mode == 'indirect':
        return ei_or_ef - neutron_mass * L1**2 / 2 / delta_t**2

In [ ]:
# -- YOUR CODE HERE --

## Fibonacci

The Fibonacci sequence is a common example for a recursive function. Here we indeed have a recursive implementation that we want to test.
This can of course be done with a number of examples that you can pick manually.

However, there is a different approach. We can also compute the Fibonacci numbers using 
$$
x_n = \frac{\varphi^n - {(1 - \varphi)}^n}{\sqrt{5}}
$$
where $\varphi \approx 1.61803399$ is the golcen ratio. You can get a high precision approximation of it from scipy:
```python
from scipy.constants import golden
```
Make sure to convert the result of the above equation to an integer or use an approximate comparison when checking against the recursive implementation.

In [ ]:
def fibonacci(n):
    """
    Return the nth Fibonacci number.
    """
    if n in (0, 1):
        return n
    return fibonacci(n - 1) + fibonacci(n - 2)

In [ ]:
# -- YOUR CODE HERE --
from scipy.constants import golden
def fib_golden(n):
    return int((golden**n - (1 - golden)**n) / np.sqrt(5))
for n in range(20):
    assert fibonacci(n) == fib_golden(n)
    
with pytest.raises(ValueError):
    fibonacci(-1)